In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# to be used as subworkflow
import nlppln
import ochre
import os

with nlppln.WorkflowGenerator(working_dir='/home/jvdzwaan/cwl-working-dir/') as wf:
    wf.load(steps_dir=ochre.cwl_path())
    print(wf.list_steps())
    
    wf.set_documentation('This workflow is meant to be used as a subworkflow.')

    gs_unnormalized = wf.add_input(gs_files='File[]')
    ocr_unnormalized = wf.add_input(ocr_files='File[]')
    language = wf.add_input(language='string')
    lowercase = wf.add_input(lowercase='boolean?')
    align_metadata = wf.add_input(align_m='string?')
    align_changes = wf.add_input(align_c='string?')
    word_mapping_name = wf.add_input(wm_name='string?')

    gs = wf.normalize_whitespace_punctuation(meta_in=gs_unnormalized, scatter=['meta_in'], scatter_method='dotproduct')
    ocr = wf.normalize_whitespace_punctuation(meta_in=ocr_unnormalized, scatter=['meta_in'], scatter_method='dotproduct')

    alignments, changes, metadata = wf.align_texts_wf(gs=gs, ocr=ocr, align_c=align_changes, align_m=align_metadata)
    
    gs_saf = wf.pattern(in_file=gs, language=language, scatter='in_file', scatter_method='dotproduct')

    mappings = wf.create_word_mappings(alignments=alignments, saf=gs_saf, lowercase=lowercase, 
                                       scatter=['alignments', 'saf'], scatter_method='dotproduct')
    merged = wf.merge_csv(in_files=mappings, name=word_mapping_name)

    wf.add_outputs(wm_mapping=merged)
    wf.save(os.path.join(ochre.cwl_path(), 'word-mapping-wf.cwl'), wd=True)

In [ ]:
# create word mappings for directory
import nlppln
import ochre
import os

with nlppln.WorkflowGenerator(working_dir='/home/jvdzwaan/cwl-working-dir/') as wf:
    wf.load(steps_dir=ochre.cwl_path())
    print(wf.list_steps())
    
    gs_dir = wf.add_input(gs_dir='Directory')
    ocr_dir = wf.add_input(ocr_dir='Directory')
    language = wf.add_input(language='string')
    lowercase = wf.add_input(lowercase='boolean?')
    align_metadata = wf.add_input(align_m='string?')
    align_changes = wf.add_input(align_c='string?')
    word_mapping_name = wf.add_input(wm_name='string?')

    gs_files = wf.ls(in_dir=gs_dir)
    ocr_files = wf.ls(in_dir=ocr_dir)

    wm_mapping = wf.word_mapping_wf(gs_files=gs_files, ocr_files=ocr_files, language=language, 
                                    lowercase=lowercase, align_c=align_changes, 
                                   align_m=align_metadata, wm_name=word_mapping_name)

    wf.add_outputs(wm_mapping=wm_mapping)
    wf.save(os.path.join(ochre.cwl_path(), 'word-mapping-dir.cwl'), pack=True)

In [ ]:
# create word mappings for test data
import nlppln
import ochre
import os

with nlppln.WorkflowGenerator(working_dir='/home/jvdzwaan/cwl-working-dir/') as wf:
    wf.load(steps_dir=ochre.cwl_path())
    print(wf.list_steps())

    gs_dir = wf.add_input(gs_dir='Directory')
    ocr_dir = wf.add_input(ocr_dir='Directory')
    language = wf.add_input(language='string')
    data_div = wf.add_input(data_div='File')
    lowercase = wf.add_input(lowercase='boolean?')
    align_metadata = wf.add_input(align_m='string?')
    align_changes = wf.add_input(align_c='string?')
    word_mapping_name = wf.add_input(wm_name='string?')

    test_gs = wf.select_test_files(datadivision=data_div, in_dir=gs_dir)
    test_ocr = wf.select_test_files(datadivision=data_div, in_dir=ocr_dir)

    wm_mapping = wf.word_mapping_wf(gs_files=test_gs, ocr_files=test_ocr, language=language, 
                                    lowercase=lowercase, align_c=align_changes, 
                                    align_m=align_metadata, wm_name=word_mapping_name)

    wf.add_outputs(wm_mapping=wm_mapping)
    wf.save(os.path.join(ochre.cwl_path(), 'word-mapping-test-files-wf.cwl'), pack=True)